In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, clear_output
import ipywidgets as widgets

class Reminder:
    def __init__(self, title, reminder_datetime):
        self.title = title
        self.reminder_datetime = reminder_datetime

    def notify(self):
        print(f"Reminder: {self.title} - Time: {self.reminder_datetime.strftime('%Y-%m-%d %H:%M')}")

class Task:
    def __init__(self, title, description="", priority="Medium", due_date=None):
        self.title = title
        self.description = description
        self.priority = priority
        self.due_date = due_date
        self.progress = 0
        self.reminders = []

    def add_reminder(self, reminder):
        self.reminders.append(reminder)

    def clear_reminders(self):
        self.reminders.clear()

class TaskManager:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        self.tasks.append(task)

    def prioritize_tasks(self):
        self.tasks.sort(key=lambda x: x.priority)

    def update_progress(self, task_title, progress):
        for task in self.tasks:
            if task.title == task_title:
                task.progress = int(progress)
                if task.progress == 100:
                    task.clear_reminders()
                    print(f"Task '{task.title}' is completed. All reminders cleared.")
                break

class TaskPlanner:
    def __init__(self):
        self.reminders = []
        self.task_manager = TaskManager()

    def create_reminder(self, task, reminder_datetime):
        reminder = Reminder(task.title, reminder_datetime)
        self.reminders.append(reminder)
        task.add_reminder(reminder)

    def view_reminders(self):
        clear_output()
        if not self.reminders:
            print("No reminders set.")
        else:
            print("\nReminders:")
            for i, reminder in enumerate(self.reminders):
                print(f"{i+1}. {reminder.title} - Time: {reminder.reminder_datetime.strftime('%Y-%m-%d %H:%M')}")
        display(back_button)

    def create_task(self, title, description="", priority="Medium", due_date=None):
        task = Task(title, description, priority, due_date)
        self.task_manager.add_task(task)
        self.schedule_reminders(task)
        clear_output()
        print("Task created successfully!")
        display(back_button)

    def view_tasks(self):
        clear_output()
        print("\nTasks:")
        self.task_manager.prioritize_tasks()
        for i, task in enumerate(self.task_manager.tasks):
            print(f"{i+1}. {task.title} - Priority: {task.priority} - Due Date: {task.due_date.strftime('%Y-%m-%d %H:%M')} - Progress: {task.progress}%")
        display(back_button)

    def update_task_progress(self, task_title, progress):
        self.task_manager.update_progress(task_title, progress)
        clear_output()
        print(f"\nProgress updated for task '{task_title}': {progress}%")
        display(back_button)

    def schedule_reminders(self, task):
        current_time = datetime.now()
        one_day_before = task.due_date - timedelta(days=1)
        one_hour_before = task.due_date - timedelta(hours=1)

        if task.progress < 100:
            if one_day_before > current_time:
                self.create_reminder(task, one_day_before)
            if one_hour_before > current_time:
                self.create_reminder(task, one_hour_before)
            self.create_reminder(task, task.due_date)

def show_main_menu():
    clear_output()
    display(menu)

def create_task_button_handler(_):
    clear_output()
    display(confirm_create_task_widget)

def confirm_create_task_button_handler(_):
    title = title_widget.value
    description = description_widget.value
    priority = priority_widget.value
    due_date = due_date_widget.value
    due_time = due_time_widget.value

    if not title:
        print("Title is required.")
        return

    if not due_date:
        print("Due date is required.")
        return

    if not due_time:
        print("Due time is required.")
        return

    try:
        due_time = datetime.strptime(due_time, "%H:%M").time()
    except ValueError:
        print("Invalid time format. Please enter time as HH:MM.")
        return

    due_datetime = datetime.combine(due_date, due_time)

    task_planner.create_task(title, description, priority, due_datetime)

def view_tasks_button_handler(_):
    task_planner.view_tasks()

def update_progress_button_handler(_):
    clear_output()
    display(confirm_update_progress_widget)

def confirm_update_progress_button_handler(_):
    task_title = task_title_widget.value
    progress = progress_widget.value
    task_planner.update_task_progress(task_title, progress)

def view_reminders_button_handler(_):
    task_planner.view_reminders()

def confirm_back_to_main_menu(_):
    show_main_menu()

create_task_button = widgets.Button(description="Create Task")
create_task_button.on_click(create_task_button_handler)

view_tasks_button = widgets.Button(description="View Tasks")
view_tasks_button.on_click(view_tasks_button_handler)

update_progress_button = widgets.Button(description="Update Task Progress")
update_progress_button.on_click(update_progress_button_handler)

view_reminders_button = widgets.Button(description="View Reminders")
view_reminders_button.on_click(view_reminders_button_handler)

back_button = widgets.Button(description="Back to Main Menu")
back_button.on_click(confirm_back_to_main_menu)

menu = widgets.VBox([
    widgets.Label("Main Menu"),
    create_task_button,
    view_tasks_button,
    update_progress_button,
    view_reminders_button
])

# Create Task Section
title_widget = widgets.Text(description="Title:")
description_widget = widgets.Textarea(description="Description:")
priority_widget = widgets.Dropdown(options=['High', 'Medium', 'Low'], description="Priority:")
due_date_widget = widgets.DatePicker(description="Due Date:")
due_time_widget = widgets.Text(description="Due Time (HH:MM):")
confirm_create_task_button = widgets.Button(description="Confirm")
confirm_create_task_button.on_click(confirm_create_task_button_handler)
confirm_create_task_widget = widgets.VBox([
    widgets.Label("Create Task"),
    title_widget,
    description_widget,
    priority_widget,
    due_date_widget,
    due_time_widget,
    confirm_create_task_button
])

# View Tasks Section
task_title_widget = widgets.Text(description="Task Title:")
progress_widget = widgets.IntSlider(min=0, max=100, step=1, description="Progress:")
confirm_update_progress_button = widgets.Button(description="Confirm")
confirm_update_progress_button.on_click(confirm_update_progress_button_handler)
confirm_update_progress_widget = widgets.VBox([
    widgets.Label("Update Task Progress"),
    task_title_widget,
    progress_widget,
    confirm_update_progress_button
])

show_main_menu()